In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Huggingface transformers 라이브러리 설치
! pip install transformers
# Install datasets
!pip install datasets
# Install sentence-transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [4]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("eenzeenee/t5-small-korean-summarization")

model = AutoModelForSeq2SeqLM.from_pretrained("eenzeenee/t5-small-korean-summarization")

In [6]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(50358, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [10]:
from torchsummary import summary

summary(model, input_size=(32, torch.tensor([1024 * [0]])))

TypeError: ignored

In [7]:
input_ids = torch.tensor([1024 * [0]])  # shape (1, 600)
model(input_ids, decoder_input_ids=input_ids)  # => no error

Seq2SeqLMOutput(loss=None, logits=tensor([[[-1.0051,  3.2642, -2.5107,  ..., -7.0331, -7.1734, -7.2485],
         [-1.0051,  3.2642, -2.5107,  ..., -7.0331, -7.1734, -7.2485],
         [-1.0051,  3.2642, -2.5107,  ..., -7.0331, -7.1734, -7.2485],
         ...,
         [-1.0051,  3.2642, -2.5107,  ..., -7.0331, -7.1734, -7.2485],
         [-1.0051,  3.2642, -2.5107,  ..., -7.0331, -7.1734, -7.2485],
         [-1.0051,  3.2642, -2.5107,  ..., -7.0331, -7.1734, -7.2485]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-0.3408,  0.6735, -0.2075,  ..., -0.6160, -0.6756,  0.7782],
          [-0.3408,  0.6735, -0.2075,  ..., -0.6160, -0.6756,  0.7782],
          [-0.3408,  0.6735, -0.2075,  ..., -0.6160, -0.6756,  0.7782],
          ...,
          [-0.3408,  0.6735, -0.2075,  ..., -0.6160, -0.6756,  0.7782],
          [-0.3408,  0.6735, -0.2075,  ..., -0.6160, -0.6756,  0.7782],
          [-0.3408,  0.6735, -0.2075,  ..., -0.6160, -0.6756,  0.7782]],

         [[ 0.0367

RuntimeError: ignored

In [7]:
# 데이터프레임 생성
data = pd.read_csv('/content/drive/MyDrive/text_summary/요약 데이터/data_train.csv', sep=',', index_col=0)

In [8]:
# 데이터 로드 및 전처리
prefix = "summarize: "

passages_before = data["passage"].tolist()
passages = []
for i in passages_before:
  input = prefix + i
  passages.append(input)
summaries = data["summary"].tolist()

In [9]:
# 데이터셋 생성
dataset = list(zip(passages, summaries))

In [10]:
dataset[0]

('summarize: 외부세계의 입장에서 북한의 거시경제 추세에 본격적인 관심을 기울이기 시작한 것은 1990년대부터였다. 무엇보다 당시 소비에트 경제가 붕괴하고 동유럽 사회주의 국가들이 무너지면서, 동유럽 국가들보다 더욱 스탈린주의에 충실한 경직적 사회주의 경제체제를 유지했던 북한의 미래가 많은 사람들의 관심을 끌 수밖에 없었기 때문이다. 이에 따라 한국을 비롯한 국제 사회에서는 1990년대 초반을 기점으로 북한의 거시경제 추세를 보다 면밀하게 관찰하려는 욕구가 증대하였다. 그런데 오늘날과 마찬가지로 당시에도 북한의 거시경제 추세를 분석하는 데 있어 가장 커다란 걸림돌은 바로 이용 가능한 통계 데이터의 부재 현상이었다. 실제로 당시까지 북한은 일체의 거시경제 데이터를 공개하지 않아, 외부세계의 입장에서는 북한경제를 분석하고 싶어도 현실적으로 이것이 불가능한 상황에 처해 있었던 것이다. 그렇다면 그간 북한경제 연구자들은 이러한 상황에 어떻게 대처하여 왔을까?',
 '1990년대부터 동유럽 사회주의 국가들이 붕괴하면서 마찬가지로 사회주의 경제체제를 유지하고 있던 북한의 거시 경제에 대한 관심이 높아졌다. 그러나 북한의 거시 경제에 대한 정보는 공개되어 있지 않아 데이터 수집에 난항을 겪을 수밖에 없었다.')

In [11]:
# 데이터로더 생성
batch_size = 32  # 배치 크기 설정
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [12]:
# 토크나이저에 특별 토큰 추가
tokenizer.add_tokens(["[CLS]", "[SEP]"])

2

In [13]:
# 모델과 토크나이저를 GPU로 이동
model.to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(50360, 512)

In [18]:
print("Tokenizer Vocab Size:", tokenizer.vocab_size)
print("Model Vocab Size:", model.config.vocab_size)

Tokenizer Vocab Size: 50258
Model Vocab Size: 50360


In [14]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8, no_deprecation_warning=True)

In [15]:
# 학습률 스케줄러 설정
num_epochs = 10  # 총 학습 에폭 수
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
#RDASS로 평가
from sentence_transformers import SentenceTransformer, util
m = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
# 학습 루프
model.train()

logging_steps = 2000  # 로깅 스텝 설정

for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        passages = batch[0][0]
        summaries = batch[1][0]

        # 토큰화 및 인코딩
        inputs = tokenizer(passages, truncation=True, padding=True, max_length=1024, return_tensors="pt")
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        labels = tokenizer(summaries, truncation=True, padding=True, max_length=256, return_tensors="pt").input_ids
        labels = labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 그라디언트 클리핑
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if step % logging_steps == 0:
            print(f"Epoch {epoch+1} / Step {step+1}, Loss: {loss.item()}")
            # 요약문 생성
            # 세부 파라미터 조정 필요
            summary_text_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                length_penalty=1.0,  # 길이에 대한 페널티, 1보다 작은 경우 더 짧은 문장 생성 유도, 1보다 클 경우 더 긴 문장 유도
                max_length=256,  # 요약문의 최대 길이 설정
                min_length=32,  # 요약문의 최소 길이 설정
                num_beams=3  # 문장 생성시 다음 단어를 탐색하는 영역의 개수
            )
            # texts
            news_text = batch[0]
            sum_text = batch[1]
            result_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

            # encode sentences into vectors
            s1 = [news_text]
            v1 = m.encode(s1)

            s2 = [sum_text]
            v2 = m.encode(s2)

            s3 = [result_text]
            v3 = m.encode(s3)

            # compute similarity between sentence vectors
            sim_a = util.cos_sim(v1, v2)
            sim_b = util.cos_sim(v2, v3)

            # RDASS
            RDASS = (sim_a+sim_b) / 2
            print('RDASS : ', RDASS.item())

Epoch 1 / Step 1, Loss: 1.4565744400024414
RDASS :  0.6315032839775085
Epoch 1 / Step 2001, Loss: 0.27814117074012756
RDASS :  0.7310658693313599
Epoch 2 / Step 1, Loss: 1.0847762823104858
RDASS :  0.7591499090194702
Epoch 2 / Step 2001, Loss: 0.7027957439422607
RDASS :  0.6292089223861694
Epoch 3 / Step 1, Loss: 0.22090652585029602
RDASS :  0.8279162645339966
Epoch 3 / Step 2001, Loss: 0.08269012719392776
RDASS :  0.8579977750778198
Epoch 4 / Step 1, Loss: 0.8225058913230896
RDASS :  0.8550958037376404
Epoch 4 / Step 2001, Loss: 0.6859175562858582
RDASS :  0.784908652305603
Epoch 5 / Step 1, Loss: 0.12822698056697845
RDASS :  0.7551586627960205
Epoch 5 / Step 2001, Loss: 0.9743970036506653
RDASS :  0.8275296092033386
Epoch 6 / Step 1, Loss: 0.23241178691387177
RDASS :  0.9007103443145752
Epoch 6 / Step 2001, Loss: 0.3768613934516907
RDASS :  0.8571387529373169
Epoch 7 / Step 1, Loss: 1.0949102640151978
RDASS :  0.7353875041007996
Epoch 7 / Step 2001, Loss: 0.9340646266937256
RDASS :  

In [ ]:
# 요약문 생성
# 세부 파라미터 조정 필요
summary_text_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    length_penalty=1.0,  # 길이에 대한 페널티, 1보다 작은 경우 더 짧은 문장 생성 유도, 1보다 클 경우 더 긴 문장 유도
    max_length=256,  # 요약문의 최대 길이 설정
    min_length=32,  # 요약문의 최소 길이 설정
    num_beams=2  # 문장 생성시 다음 단어를 탐색하는 영역의 개수
)

In [ ]:
news_text = batch[0]
print(news_text)

In [ ]:
sum_text = batch[1]
print(sum_text)

('정부는「정부 3. 0」의 새로운 행정 패러다임을 제시, 공공정보의 공개와 활용을 확대하고 공공정보의 개방·공유로 민간 활용을 높이고 부처 간 상호 소통, 협력하여 국정과제에 대한 추진동력을 확보 한다. ', '본 연구는 인터넷 상의 표현 방식 교육의 구체적인 교육 내용과 방법을 설계하기 위한 기초 자료 조사 연구이다. 인터넷 이용 실태와 사이버 문학, 통신 언어, 인터넷 교육의 필요성에 관한 인식에 있어서 교사와 학생 두 집단은 상당한 차이가 있었다. 교아와 학생의 정보 격차는 인터넷의 의사소통 방식을 교육하는 데 있어 제한점이 될 수 있으므로 새로운 교육 내용을 받아들이고 시행할 준비가 되지 않& 교사에게는 재교육의 기회가 필요하다.', '이 논문에서는 중국유학생들과한국대학생들의 인터넷 쇼핑몰 고객만족도에 관해 비교 분석해 어떤 차이가 있는지 분석하고 전역적 최적의 해를구하는 다중 LS-SVM을 이용해 중국 유학생들의 고객만족도 모형을 분석한다.', '단축이방성 박막들과 등방성 박막들이 코팅되어 있는 시료에 비스듬히 입사한 빛의 유효반사계수 표현들을 유도했다.', '이 연구의 목적은 한국선도의 역사적 전개속에서 女神仙의 존재를 추적하여 그 계보를 재구성함으로써 한국선도사에서 여신선의 의미와 위상을 이해하려는 것이다. 그동안 한국의 철학계에서는 유교와 불교에 대한 연구가 주류를 이루고 있었을 뿐, 도교 연구가 본격화된 것은 얼마되지 않았다. 특히 한국의 선도라는 영역에 대한 관심이 일어난 것 자체가 최근 10년 내의 일이다.', '이런 현실에서 정권 초창기에 과거 유산 정리와 광역교통기구 설립이 필요하다. 기구를 출범하고 예산 배정과 민간 기업 투자 등의 진행이 시작돼야 한다. 현 수도권의 복잡한 교통수단의 대책 제시와 통합운영도 필요하다.', '이 연구는 「규장각지」의 완성 과정과 제도 정비를 통한 규장각제도가 정조에서 순조대로 어떻게 전승되었는지를 보고자 한다. 선행연구를 통해 축적되어 있는 부분을 제외하고 초초본, 재초본 등 거듭된 수정 보완의 과정과

In [ ]:
result_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
print(result_text)

정부는 급격한 사회문제와 지식정보사회로의 전환으로 정부-국민 간 관계 변화함에 따라 「정부 3.0」의 새로운 행정 패러다임을 제시했다.


In [ ]:
#RDASS로 평가
from sentence_transformers import SentenceTransformer, util
m = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# encode sentences into vectors
s1 = [news_text]
v1 = m.encode(s1)

s2 = [sum_text]
v2 = m.encode(s2)

s3 = [result_text]
v3 = m.encode(s3)

print(v1.shape, v2.shape, v3.shape)

# compute similarity between sentence vectors
sim_a = util.cos_sim(v1, v2)
sim_b = util.cos_sim(v2, v3)

print(sim_a)
print(sim_b)

# RDASS
RDASS = (sim_a+sim_b) / 2
print('RDASS : ', RDASS.item())

(1, 768) (1, 768) (1, 768)
tensor([[0.8751]])
tensor([[0.6759]])
RDASS :  0.7754775285720825


In [ ]:
# 모델 저장하기
torch.save(model, '/content/drive/MyDrive/text_summary/단우/model/Ke-T5_FineTuned_small.pt')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer, util

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("eenzeenee/t5-small-korean-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("eenzeenee/t5-small-korean-summarization")

# Load and preprocess the data
data = pd.read_csv('/content/drive/MyDrive/text_summary/요약 데이터/data_train.csv', sep=',', index_col=0)
prefix = "summarize: "
passages_before = data["passage"].tolist()
passages = []
for i in passages_before:
    input = prefix + i
    passages.append(input)
summaries = data["summary"].tolist()
dataset = list(zip(passages, summaries))

# Create the data loader
batch_size = 128
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Add special tokens to the tokenizer
tokenizer.add_tokens(["[CLS]", "[SEP]"])

# Move the model to the device
model.to(device)
model.resize_token_embeddings(len(tokenizer))

# Split the model into front and back parts
front_layers = model.encoder
back_layers = model.lm_head

# Freeze the parameters of the front layers
for param in front_layers.parameters():
    param.requires_grad = False

# Define optimizer and scheduler for the back layers
optimizer = AdamW(back_layers.parameters(), lr=2e-5, eps=1e-8, no_deprecation_warning=True)
num_epochs = 10
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Initialize the SentenceTransformer for RDASS evaluation
m = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# Training loop
model.train()

logging_steps = 2000

for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        passages = batch[0][0]
        summaries = batch[1][0]

        # Tokenization and encoding
        inputs = tokenizer(passages, truncation=True, padding=True, max_length=1024, return_tensors="pt")
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)
        labels = tokenizer(summaries, truncation=True, padding=True, max_length=256, return_tensors="pt").input_ids
        labels = labels.to(device)

        # Forward pass and loss calculation for back layers only
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backpropagation and optimization for back layers only
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(back_layers.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        if step % logging_steps == 0:
            print(f"Epoch {epoch+1} / Step {step+1}, Loss: {loss.item()}")
            # 요약문 생성
            # 세부 파라미터 조정 필요
            summary_text_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                length_penalty=1.0,  # 길이에 대한 페널티, 1보다 작은 경우 더 짧은 문장 생성 유도, 1보다 클 경우 더 긴 문장 유도
                max_length=256,  # 요약문의 최대 길이 설정
                min_length=32,  # 요약문의 최소 길이 설정
                num_beams=2  # 문장 생성시 다음 단어를 탐색하는 영역의 개수
            )
            # texts
            news_text = batch[0]
            sum_text = batch[1]
            result_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

            # encode sentences into vectors
            s1 = [news_text]
            v1 = m.encode(s1)

            s2 = [sum_text]
            v2 = m.encode(s2)

            s3 = [result_text]
            v3 = m.encode(s3)

            # compute similarity between sentence vectors
            sim_a = util.cos_sim(v3, v1)
            sim_b = util.cos_sim(v3, v2)

            # RDASS
            RDASS = (sim_a+sim_b) / 2
            print('RDASS : ', RDASS.item())

# Save the model
torch.save(model, '/content/drive/MyDrive/text_summary/단우/model/Ke-T5_Trans_small.pt')